In [1]:
import pandas as pd
import numpy as np
import pytz, sys,os
import plotly.express as px

sys.path.append(os.getcwd())

from datetime import datetime as dt
from datetime import timezone
from dateutil import tz
from src.config import SUBSCRIPTION_KEY, CUSTOMER_ID
from PF_API_Shared import get_all_plants, get_plant_devices, Dispatch,TimeInterval
from API_Intergration import plant_tz


In [2]:

plant = 'Latitude'
start = dt(2022,9,1)
end = dt(2022,9,30)


In [3]:
dev_tags = [("Combiner", ["DC_CURRENT"]),
            ("Inverter", ["DC_CURRENT"]),
            ("Sensor Pyranometer POA", ["IRRADIANCE_POA"]),
                ("Meter",["AC_POWER"])]
  
rq_tags = []

plants = get_all_plants(SUBSCRIPTION_KEY, CUSTOMER_ID)  
plant_id = plants["id"][plants["name"] == plant].values[0]
dispatch = Dispatch(SUBSCRIPTION_KEY, '2096')
devices = get_plant_devices(SUBSCRIPTION_KEY,CUSTOMER_ID,plant_id)



In [4]:
for dtype, tags in dev_tags:
    dtype_devices = devices[devices['type']==dtype]
    
    df = dispatch.fetch_data(dtype_devices['id'], 
                                tags, 
                                query_range=TimeInterval(starttime=start,endtime=end))
    rq_tags.append(df)
inv_data = pd.concat(rq_tags, axis=1)
data = inv_data.copy(deep=True)

In [5]:
ident = [id for site, type, id, name in data.columns]
data.columns = ident


In [6]:
data = data.set_index([data.index])
data.index = pd.to_datetime(data.index)
data = data[~data.index.duplicated(keep='first')]
_tz = plant_tz(plant)
data.index = data.index.tz_convert(_tz)


In [7]:
"""For Filtered data, the data is filtered by the following parameters
        a. POA > 100
        b. (Meter Power > 1.5 MW) & (Meter Power < 25 MW)
        c. Hour > 12 (For shading purposes)
        d. All CBX's < 0 = 0
"""

data.loc[:, "Meter_Power"] = data.filter(regex="MTR").median(axis=1)
data.loc[:, "POA"] = data.filter(regex="MET").median(axis=1)

curtailment_limit = 499999

new_df = data.between_time("7:00", "17:00")
new_df = data[(data.loc[:,"POA"] > 100)&(data.loc[:,"Meter_Power"]<curtailment_limit*0.99)]
new_df[new_df<=0.00] = 0
day_df = new_df.resample("1D").sum()

day_df


C:\Users\dpinales\AppData\Local\Temp\ipykernel_11932\4223233658.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[new_df<=0.00] = 0
C:\Users\dpinales\AppData\Local\Temp\ipykernel_11932\4223233658.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[new_df<=0.00] = 0


,2096.05.008.BLK01.PAD01.INV01.CMB01,2096.05.008.BLK01.PAD01.INV01.CMB02,2096.05.008.BLK01.PAD01.INV01.CMB03,2096.05.008.BLK01.PAD01.INV01.CMB04,2096.05.008.BLK01.PAD01.INV01.CMB05,2096.05.008.BLK01.PAD01.INV01.CMB06,2096.05.008.BLK01.PAD01.INV01.CMB07,2096.05.008.BLK01.PAD01.INV01.CMB08,2096.05.008.BLK01.PAD01.INV01.CMB09,2096.05.008.BLK01.PAD01.INV01.CMB10,...,2096.05.008.BLK01.PAD03.INV01,2096.05.008.BLK01.PAD04.INV01,2096.05.008.BLK01.PAD05.INV01,2096.05.008.MET07.PYR01,2096.05.008.MET08.PYR01,2096.05.008.MET09.PYR01,2096.05.008.METREF01.PYR01,2096.05.008.SWG01.MTR01,Meter_Power,POA
Timestamp,,,,,,,,,,,,,,,,,,,,,
2022-09-01 00:00:00-05:00,16209.912796,16667.486986,13685.042357,14377.170566,17854.835800,15565.086924,19051.136515,19294.119808,16990.815968,16917.883578,...,307163.573262,323066.990830,0.000000,0.0,0.0,0.0,123780.041066,1.197288e+06,1.197288e+06,123780.041066
2022-09-02 00:00:00-05:00,10656.373215,11075.146538,6821.917651,9433.000198,11752.627438,10217.026828,12482.100086,12600.606537,11131.568372,11011.074669,...,122207.929179,133663.310146,0.000000,0.0,0.0,0.0,107671.804784,7.887039e+05,7.887039e+05,107671.804784
2022-09-03 00:00:00-05:00,12106.611793,12427.039151,12364.038649,10575.029019,13102.819156,11367.787075,13833.328186,13956.920159,12266.579211,12169.493483,...,153552.099651,162294.278852,0.000000,0.0,0.0,0.0,90402.011861,8.678507e+05,8.678507e+05,90402.011861
2022-09-04 00:00:00-05:00,7887.112612,8404.037485,8232.962408,7066.242498,8814.199814,7667.165277,9333.145162,9363.955250,8278.193972,8187.094549,...,137042.751064,126559.022674,0.000000,0.0,0.0,0.0,72078.402158,5.987760e+05,5.987760e+05,72078.402158
2022-09-05 00:00:00-05:00,11978.325361,12448.230098,12329.288435,10467.530376,12917.793086,11198.735160,13778.125838,13808.766500,12130.036046,11978.304730,...,149914.017283,169032.016203,0.000000,0.0,0.0,0.0,72691.562580,8.719814e+05,8.719814e+05,72691.562580
2022-09-06 00:00:00-05:00,9338.300736,9780.759672,9625.068106,8202.829489,10154.753255,8863.660502,10724.851627,10915.309431,9686.806274,9567.484401,...,137866.716863,143520.050034,0.000000,0.0,0.0,0.0,92953.349975,6.896814e+05,6.896814e+05,92953.349975
2022-09-07 00:00:00-05:00,12587.498308,12945.855272,12951.862387,11064.176679,13784.441766,12084.159170,14887.185803,14978.680331,13097.032188,12983.608655,...,148405.369349,192568.713038,74363.833979,0.0,0.0,0.0,85115.611367,9.892956e+05,9.892956e+05,85115.611367
2022-09-08 00:00:00-05:00,17417.720755,18026.424945,18245.391993,15491.825650,19205.503817,16787.262297,20805.930841,20952.365150,18399.115645,18276.606120,...,237066.669115,278729.345958,264594.584652,0.0,0.0,0.0,110278.770920,1.585860e+06,1.585860e+06,110278.770920
2022-09-09 00:00:00-05:00,7745.529565,8057.643744,7951.403692,6798.148113,8449.626626,7358.644766,9015.290140,9308.394514,8317.015878,8247.109063,...,137562.038563,135450.539627,133860.296755,0.0,0.0,0.0,47548.233401,7.155102e+05,7.155102e+05,47548.233401


In [8]:
path = "Project_Metadata_tables.xlsx"

dc_meta = pd.read_excel(path, sheet_name="Combiner_table").filter(items= ['plant_name', 'string_count','DC Rating Power kW', 'Module IMP'], axis=1)


In [11]:
day_df = day_df.filter(regex="CMB")

dc_issues = dc_meta[dc_meta["plant_name"]==plant]  
dc_rating = [i for i in dc_issues['DC Rating Power kW'].values]

relative_dc = []

max_dc = max(dc_rating)

for i in dc_rating:
    dc = (i/max_dc)
    relative_dc.append(dc)

string_count = [s for s in dc_issues['string_count']]

cmb_relative = day_df.div(relative_dc)

cmb_relative.loc[:,'norm'] = cmb_relative.apply(pd.Series.nlargest, axis = 1, n = 5).median(axis = 1)
t = cmb_relative.div(cmb_relative['norm'], axis = 0).clip(lower = 0, upper = 1)
t = t.iloc[:,:-1]
t = t.fillna(0)

down_strings = (string_count*(1-t)).round()
cbx_index = list(down_strings.index)
d_strings = list(down_strings.columns)
down_strings

,2096.05.008.BLK01.PAD01.INV01.CMB01,2096.05.008.BLK01.PAD01.INV01.CMB02,2096.05.008.BLK01.PAD01.INV01.CMB03,2096.05.008.BLK01.PAD01.INV01.CMB04,2096.05.008.BLK01.PAD01.INV01.CMB05,2096.05.008.BLK01.PAD01.INV01.CMB06,2096.05.008.BLK01.PAD01.INV01.CMB07,2096.05.008.BLK01.PAD01.INV01.CMB08,2096.05.008.BLK01.PAD01.INV01.CMB09,2096.05.008.BLK01.PAD01.INV01.CMB10,...,2096.05.008.BLK01.PAD05.INV01.CMB09,2096.05.008.BLK01.PAD05.INV01.CMB10,2096.05.008.BLK01.PAD05.INV01.CMB11,2096.05.008.BLK01.PAD05.INV01.CMB12,2096.05.008.BLK01.PAD05.INV01.CMB13,2096.05.008.BLK01.PAD05.INV01.CMB14,2096.05.008.BLK01.PAD05.INV01.CMB15,2096.05.008.BLK01.PAD05.INV01.CMB16,2096.05.008.BLK01.PAD05.INV01.CMB17,2096.05.008.BLK01.PAD05.INV01.CMB18
Timestamp,,,,,,,,,,,,,,,,,,,,,
2022-09-01 00:00:00-05:00,7.0,4.0,23.0,3.0,4.0,3.0,4.0,3.0,2.0,2.0,...,130.0,131.0,125.0,119.0,126.0,130.0,133.0,118.0,134.0,121.0
2022-09-02 00:00:00-05:00,7.0,3.0,45.0,3.0,4.0,3.0,5.0,4.0,2.0,4.0,...,130.0,135.0,124.0,119.0,126.0,130.0,133.0,118.0,133.0,120.0
2022-09-03 00:00:00-05:00,4.0,1.0,2.0,2.0,3.0,3.0,5.0,4.0,3.0,3.0,...,131.0,135.0,123.0,119.0,126.0,130.0,132.0,118.0,133.0,120.0
2022-09-04 00:00:00-05:00,8.0,1.0,3.0,3.0,3.0,3.0,5.0,4.0,3.0,4.0,...,128.0,136.0,121.0,116.0,123.0,128.0,130.0,116.0,131.0,117.0
2022-09-05 00:00:00-05:00,6.0,2.0,3.0,3.0,5.0,5.0,6.0,5.0,4.0,6.0,...,130.0,134.0,121.0,114.0,121.0,126.0,128.0,114.0,129.0,115.0
2022-09-06 00:00:00-05:00,6.0,1.0,2.0,3.0,4.0,3.0,6.0,4.0,2.0,3.0,...,128.0,135.0,122.0,117.0,124.0,129.0,132.0,117.0,132.0,119.0
2022-09-07 00:00:00-05:00,5.0,2.0,2.0,2.0,3.0,2.0,2.0,1.0,1.0,2.0,...,104.0,112.0,93.0,79.0,85.0,87.0,89.0,78.0,90.0,79.0
2022-09-08 00:00:00-05:00,8.0,4.0,3.0,3.0,5.0,4.0,4.0,3.0,2.0,3.0,...,44.0,118.0,30.0,9.0,9.0,9.0,6.0,5.0,12.0,4.0
2022-09-09 00:00:00-05:00,11.0,6.0,8.0,7.0,9.0,8.0,10.0,6.0,3.0,4.0,...,17.0,109.0,10.0,7.0,25.0,14.0,11.0,9.0,15.0,5.0


In [43]:

fig = px.imshow(down_strings, labels=dict(x="Downstrings", y="Dates", color="String Count"), x=d_strings, y=cbx_index, height=900, width=1500, text_auto=True, aspect="auto")
fig.update_xaxes(side="top")
fig.update_layout(yaxis_nticks=len(cbx_index))
fig.show()

KeyboardInterrupt: 

In [ ]:
e_downstrings = sum(int(down_strings[:]))
total_string = sum(int(string_count))
ratio_strings = e_downstrings/total_string

print(f"The estimated plants DC outage: {ratio_strings}")

In [ ]:
sum_of_cbx = cbx_data["Sum_of_CBX"]
sum_of_inv = inv_df["Sum_of_inv"]

def cmb_summation(a,b):
    total = (a/b)
    return total


In [ ]:
# cbx_data.to_csv(f"{plant} - August_Daily_Derates.csv")
cbx_data.to_csv(f"{plant} - API Data.csv")